In [ ]:
import re
import random
from collections import defaultdict

#ADDS NOISE TO THE DURATIONS OF ROBOT ACTIONS

def parse_and_add_noise_blocks(blocks, stddev_fraction=0.05, seed=36):
    random.seed(seed)
    action_durations = defaultdict(list)
    raw_trajectories = []
    regex = re.compile(r"r(\d+)\((\d+(?:\.\d+)?)\):p(\d+)\(([^)]+)\),p(\d+)\(([^)]+)\)")

    for block in blocks:
        lines = block.strip().split("\n")
        trajectory = []

        for line in lines:
            actions = line.strip().split("-")
            timestep = []

            for action in actions:
                match = regex.match(action.strip())
                if match:
                    r_id, duration_str, p_start, y_start_str, p_end, y_end_str = match.groups()
                    duration = float(duration_str)
                    p_start = int(p_start)
                    p_end = int(p_end)

                    action_durations[(p_start, p_end)].append(duration)

                    timestep.append({
                        "robot_id": int(r_id),
                        "duration": duration,
                        "p_start": p_start,
                        "y_start_str": y_start_str,
                        "p_end": p_end,
                        "y_end_str": y_end_str
                    })
                else:
                    print(f"Warning: Could not parse action: {action}")
            trajectory.append(timestep)
        raw_trajectories.append(trajectory)

    # Compute stats
    action_stats = {
        key: {
            "mean": sum(durations) / len(durations),
            "stddev": (stddev_fraction * (sum(durations) / len(durations)))
        }
        for key, durations in action_durations.items()
    }

    # Add noise
    final_trajectories = []
    for trajectory in raw_trajectories:
        new_trajectory = []
        for timestep in trajectory:
            new_timestep = []
            for action in timestep:
                key = (action["p_start"], action["p_end"])
                mean = action_stats[key]["mean"]
                stddev = action_stats[key]["stddev"]
                noisy_duration = max(0.01, random.normalvariate(mean, stddev))

                new_timestep.append({
                    "robot_id": action["robot_id"],
                    "duration": round(noisy_duration, 2),
                    "p_start": action["p_start"],
                    "y_start": [int(label[1:]) for label in action["y_start_str"].split(',')],
                    "p_end": action["p_end"],
                    "y_end": [int(label[1:]) for label in action["y_end_str"].split(',')]
                })
            new_trajectory.append(new_timestep)
        final_trajectories.append(new_trajectory)

    return final_trajectories

def format_trajectory(traj):
    lines = []
    for timestep in traj:
        line = "-".join([
            f"r{a['robot_id']}({a['duration']:.2f}):p{a['p_start']}(y{','.join(map(str, a['y_start']))}),p{a['p_end']}(y{','.join(map(str, a['y_end']))})"
            for a in timestep
        ])
        lines.append(line)
    return "\n".join(lines)

def save_trajectories_to_file(trajectories, filename):
    with open(filename, 'w') as f:
        for trajectory in trajectories:
            lines = []
            for timestep in trajectory:
                actions_str = []
                for action in timestep:
                    r_id = action["robot_id"]
                    duration = f"{action['duration']:.2f}"
                    p_start = action["p_start"]
                    y_start = ",".join(f"y{y}" for y in action["y_start"])
                    p_end = action["p_end"]
                    y_end = ",".join(f"y{y}" for y in action["y_end"])

                    action_str = f"r{r_id}({duration}):p{p_start}({y_start}),p{p_end}({y_end})"
                    actions_str.append(action_str)

                line = "-".join(actions_str)
                lines.append(line)

            f.write("\n".join(lines) + "\n\n")  # Double newline between trajectories


# Full pipeline
with open("your_path.txt", 'r') as f:
    content = f.read()

blocks = content.strip().split("\n\n")

first = blocks[:1000]
second = blocks[1000:2000]

all_blocks = [first, second]


all_noisy_trajectories = []

for block in all_blocks:
    noisy_block = parse_and_add_noise_blocks(block, stddev_fraction=0.1)
    all_noisy_trajectories.extend(noisy_block)  # Add to the combined list

save_trajectories_to_file(all_noisy_trajectories, "your_path_noisy.txt")

